In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('new_sms_train.csv')
test_df = pd.read_csv('new_sms_test.csv')


In [2]:
train_df.head()

,text,label
0,5 Reasons You Need to Join InstaSpam Now!\n\n1...,spam
1,Win A FREE iPhone X Now!! Click Here To Enter!...,spam
2,CHECK OUT THESE AMAZING DEALS! GET RICH QUICK ...,spam
3,Urgent update!!1!!1!1 OMG 😱😱😱\nYou won't belie...,spam
4,Get rich quick with our amazing new investment...,spam


In [3]:
test_df.head()

,text,label
0,LeTs tAlK AbouT DisCOuNtS aNd DeAlS! WhO HaS T...,spam
1,Waaassssupppppp!!!!!! Have you checked out the...,spam
2,$$$MAKE $$$MONEY$$$ FAST!!!$$$ No scams!$$$ \n...,spam
3,Get rich quick with our amazing new product!!!...,spam
4,Are you tired of being single? Join our websit...,spam


In [4]:
len(train_df)

9950

In [5]:
len(test_df)

950

In [6]:
# Assuming train_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_train = train_df[train_df['label'] == 'spam']
not_spam_samples_train = train_df[train_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_train = min(len(spam_samples_train), len(not_spam_samples_train))
max_total_samples_train = 3000  # Maximum total number of samples

# Calculate the number of samples to select for the training set, ensuring it doesn't exceed the maximum
num_samples_train = min(num_samples_per_class_train * 2, max_total_samples_train)

# Sample an equal number of instances from each class
sampled_spam_train = spam_samples_train.sample(n=num_samples_train // 2, random_state=42)
sampled_not_spam_train = not_spam_samples_train.sample(n=num_samples_train // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_train_df = pd.concat([sampled_spam_train, sampled_not_spam_train])

# Shuffle the dataframe to mix spam and not spam samples
sampled_train_df = sampled_train_df.sample(frac=1, random_state=42)

print(len(sampled_train_df))


3000


In [7]:
# Assuming test_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_test = test_df[test_df['label'] == 'spam']
not_spam_samples_test = test_df[test_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_test = min(len(spam_samples_test), len(not_spam_samples_test))
max_total_samples = 800  # Maximum total number of samples

# Calculate the number of samples to select for test set, ensuring it doesn't exceed the maximum
num_samples_test = min(num_samples_per_class_test * 2, max_total_samples)

# Sample an equal number of instances from each class
sampled_spam_test = spam_samples_test.sample(n=num_samples_test // 2, random_state=42)
sampled_not_spam_test = not_spam_samples_test.sample(n=num_samples_test // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_test_df = pd.concat([sampled_spam_test, sampled_not_spam_test])

# Shuffle the dataframe to mix spam and not spam samples
sampled_test_df = sampled_test_df.sample(frac=1, random_state=42)

print(len(sampled_test_df))


800


In [8]:
# Define the mapping
label_mapping = {"spam": 0, "not_spam": 1}

# Apply the mapping to the 'label' column
train_df['label'] = train_df['label'].map(label_mapping)
test_df['label'] = test_df['label'].map(label_mapping)


In [9]:
train_df.head()

,text,label
0,5 Reasons You Need to Join InstaSpam Now!\n\n1...,0
1,Win A FREE iPhone X Now!! Click Here To Enter!...,0
2,CHECK OUT THESE AMAZING DEALS! GET RICH QUICK ...,0
3,Urgent update!!1!!1!1 OMG 😱😱😱\nYou won't belie...,0
4,Get rich quick with our amazing new investment...,0


In [10]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

# Assuming train_df and test_df are your training and testing DataFrames respectively

# Tokenization for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

# Encoding as numbers for training dataset with limited tokenization
encoded_train_texts = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')
X_train_padded = pad_sequences(encoded_train_texts['input_ids'], padding='post', maxlen=128)
y_train = train_df['label'].to_numpy()

# Encoding as numbers for testing dataset with limited tokenization
encoded_test_texts = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')
X_test_padded = pad_sequences(encoded_test_texts['input_ids'], padding='post', maxlen=128)
y_test = test_df['label'].to_numpy()

# Building the DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compile the model (define optimizer, loss, and metric)

optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)  # Adjust the learning rate as needed
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model with early stopping for 1 epoch
history = model.fit(
    {'input_ids': X_train_padded, 'attention_mask': encoded_train_texts['attention_mask']},
    y_train,
    epochs=1,
    batch_size=64,
    validation_split=0.3,
    callbacks=[early_stopping]
)

# Evaluating the model on the testing dataset
eval_result = model.evaluate(
    {'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']},
    y_test
)
print(f"Test accuracy: {eval_result[1]}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 


30/30 [==============================] - 49s 2s/step - loss: 0.4286 - accuracy: 0.7811
Test accuracy: 0.7810526490211487


In [12]:
from sklearn.metrics import classification_report

# Predicting on the test set
y_pred = model.predict({'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']})
y_pred = np.argmax(y_pred.logits, axis=1)

# Generating the classification report
report = classification_report(y_test, y_pred)
print(report)


30/30 [==============================] - 53s 2s/step
              precision    recall  f1-score   support

           0       0.71      1.00      0.83       500
           1       1.00      0.54      0.70       450

    accuracy                           0.78       950
   macro avg       0.85      0.77      0.76       950
weighted avg       0.85      0.78      0.77       950

